In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import numpy as np
from gensim.models import Word2Vec
import logging
from sklearn.manifold import TSNE
import pandas as pd

In [6]:
from utils.utils import compute_gap, preprocessing, extract_tfidf_keywords
StopWords = set(stopwords.words('english'))
porter = PorterStemmer()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Raw = [line.strip('\n') for line in open('/home/hamish/Documents/datasets/1.Raw Input.txt').readlines()]

True_cluster= pd.read_excel('/home/hamish/Documents/datasets/5.True Label (Ground Truth).xlsx',header=None)

In [7]:
StopWords


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [8]:
import nltk
nltk.download('punkt')
Cleaned_Corpus = [preprocessing(line) for line in Raw]
TF_Words = extract_tfidf_keywords(Cleaned_Corpus, 20)

[nltk_data] Downloading package punkt to /home/hamish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NameError: name 'StopWords' is not defined

In [ ]:
Raw[0]

In [ ]:
TF_Words[0]

In [ ]:
model = Word2Vec([word_tokenize(line) for line in Cleaned_Corpus], size=100, window=5, min_count=1, workers=4)
doc_vec = []
for f in TF_Words:
    doc_vec.append(np.mean(model[f], axis=0))
X = pd.DataFrame(doc_vec)
#X.to_excel('C:\\Users\\Owen\\Desktop\\4470 TF-IDF Filtered Version.xlsx')

In [ ]:
#CBOW model
    # train model
CBOW_model = Word2Vec(TF_Words, min_count=1,size=100)  #CBOW
#model = Word2Vec(texts, min_count=1,size=100,sg=1) #SG
    # summarize the loaded model
print(model)

#old model:Word2Vec(vocab=35455, size=50, alpha=0.025)

In [ ]:
CBOW_model.wv.index2entity[50:]

In [ ]:
# Print results 
print("Cosine similarity between 'features' " + "and 'functions' - CBOW : "
      , CBOW_model.wv.similarity('features', 'functions')) 
print("Cosine similarity between 'science' " + "and 'technology' - CBOW : "
      , CBOW_model.wv.similarity('science', 'technology'))

In [ ]:
CBOW_model.most_similar('features')[:10]

In [ ]:
X

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=2, n_iter=1000)
digits_proj = tsne.fit_transform(X)

In [ ]:
digits_proj

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.scatter(digits_proj[:,0], digits_proj[:,1])
plt.show()


In [ ]:
#MeanShift
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=500)
print(bandwidth)
# 计算mean shift向量并进行聚类
ms = MeanShift(bandwidth=0.2, bin_seeding=True)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)

In [ ]:
unique, counts = np.unique(labels, return_counts=True)
dict(zip(unique, counts))

In [ ]:
#DBSCAN
import numpy as np

from sklearn.cluster import DBSCAN
#X = matrix_array
db = DBSCAN(eps=0.1, min_samples=5).fit(X)

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
unique, counts = np.unique(labels, return_counts=True)
dict(zip(unique, counts))

In [ ]:
unique

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt

objects = unique
y_pos = unique
performance =counts

plt.bar(objects, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('count')
plt.xlabel('clusters')
plt.title('DBSCAN clusters distribution')

for i in range(len(counts)):
    plt.annotate(str(performance[i]), xy=(objects[i],performance[i]))

plt.show()

In [ ]:
y_pos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

tsne = TSNE(n_components=2, verbose=1, perplexity=2, n_iter=1000)
plt.figure(figsize=(60,10))
for k in range(1,15):
    kmeans = KMeans(n_clusters=k)
    a = kmeans.fit_predict(X)
    plt.subplot(1,15,k)              #subplot( numRows, numCols,  plotNum)
    
    digits_proj = tsne.fit_transform(X)
    plt.scatter(digits_proj[:,0], digits_proj[:,1], c=a)
    #plt.scatter(x[:, 0], x[:, 1], c=a)
    plt.xlabel('k='+str(k))
plt.tight_layout()
plt.show()

In [ ]:
#elbow method
from scipy.spatial.distance import cdist

#plt.plot()
#X = np.array(list(zip(x1, x2))).reshape(len(x1), 2)
plt.figure(figsize=(10,10))
colors = ['b', 'g', 'r']
markers = ['o', 'v', 's']

# k means determine k
distortions = []
K = range(1,20)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')

plt.show()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
matrix_array=X.to_numpy()
gap, reference_inertia, ondata_inertia = compute_gap(KMeans(),matrix_array)


#plt.plot(range(1, 20+1), reference_inertia,'-o', label='abc')
plt.plot(range(1, 20+1), ondata_inertia,
         '-o', label='data')
plt.xlabel('k')
plt.ylabel('log(inertia)')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 6))
#for k in range(1,13):
kmeans = KMeans(n_clusters=10)
a = kmeans.fit_predict(X)
#    plt.subplot(1,13,k)              #subplot( numRows, numCols,  plotNum)
tsne = TSNE(n_components=2, verbose=1, perplexity=2, n_iter=1000)
digits_proj = tsne.fit_transform(X)
plt.scatter(digits_proj[:,0], digits_proj[:,1], c=a)
plt.xlabel('k='+str(10))
#plt.tight_layout()
plt.show()

In [ ]:
unique, counts = np.unique(a, return_counts=True)
dict(zip(unique, counts))

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

objects = unique
y_pos = unique
performance =counts

plt.bar(objects, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('count')
plt.xlabel('clusters')
plt.title('K-means clustering')

for i in range(len(counts)):
    plt.annotate(str(performance[i]), xy=(objects[i],performance[i]))

plt.show()

In [ ]:
a[:50]

In [ ]:
True_cluster_arr=True_cluster.to_numpy()
True_cluster_arr=np.squeeze(True_cluster_arr)

In [ ]:
a.shape

In [ ]:
True_cluster_arr.shape

In [ ]:
True_cluster_arr[:10]

In [ ]:
a[:10]

In [ ]:
#keman k=10
jaccard(a[:10],True_cluster_arr[:10])

In [ ]:
jaccard(a,True_cluster_arr)

In [ ]:
#FM
from sklearn.metrics.cluster import fowlkes_mallows_score
fowlkes_mallows_score(a , True_cluster_arr)

In [ ]:
#11-a
#01-b
#10-c

In [ ]:
FM(a[:10],True_cluster_arr[:10])

In [ ]:
FM(a,True_cluster_arr)

In [ ]:
#11-a
#01-b
#10-c

In [ ]:
F1(a,True_cluster_arr)

In [ ]:
F1(a[:10],True_cluster_arr[:10])

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk
matrix_array=X.to_numpy()
NUM_CLUSTERS=10
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(matrix_array, assign_clusters=True)
print (assigned_clusters)


In [ ]:
jaccard(assigned_clusters[:10],True_cluster_arr[:10])

In [ ]:
### Quadratic-time MMD with Gaussian RBF kernel
from __future__ import division
import numpy as np
import theano.tensor as T
def rbf_mmd2(X, Y, sigma=0, biased=True):
    
    gamma = 1 / (2 * sigma**2)

    XX = T.dot(X, X.T)
    XY = T.dot(X, Y.T)
    YY = T.dot(Y, Y.T)

    X_sqnorms = T.diagonal(XX)
    Y_sqnorms = T.diagonal(YY)

    K_XY = T.exp(-gamma * (
            -2 * XY + X_sqnorms[:, np.newaxis] + Y_sqnorms[np.newaxis, :]))
    K_XX = T.exp(-gamma * (
            -2 * XX + X_sqnorms[:, np.newaxis] + X_sqnorms[np.newaxis, :]))
    K_YY = T.exp(-gamma * (
            -2 * YY + Y_sqnorms[:, np.newaxis] + Y_sqnorms[np.newaxis, :]))

    if biased:
        mmd2 = K_XX.mean() + K_YY.mean() - 2 * K_XY.mean()
    else:
        m = K_XX.shape[0]
        n = K_YY.shape[0]

        mmd2 = ((K_XX.sum() - m) / (m * (m - 1))
              + (K_YY.sum() - n) / (n * (n - 1))
              - 2 * K_XY.mean())
    
    return mmd2

In [ ]:
def get_MMD(X,Y):
    import theano
    input1=np.reshape(X,(100,1))
    input2=np.reshape(Y,(100,1))
    Xth, Yth = T.matrices('X', 'Y')
    sigmath = T.scalar('sigma')
    fn = theano.function([Xth, Yth, sigmath],rbf_mmd2(Xth, Yth, sigma=sigmath))
    mmd2 = fn(input1,input2, 1)
    print(mmd2)
    return float(mmd2)

In [ ]:
get_MMD(matrix_array[0],matrix_array[1])

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk
matrix_array=X.to_numpy()
NUM_CLUSTERS=6
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(matrix_array, assign_clusters=True)
print (assigned_clusters)

In [ ]:
unique, counts = np.unique(assigned_clusters, return_counts=True)
dict(zip(unique, counts))

In [ ]:
jaccard(assigned_clusters,True_cluster_arr)

In [ ]:
TF02= pd.read_excel('L:\\09052020Topic 2 Dataset\\4.TF-IDF Filtered Version.xlsx')

In [ ]:
TF02

In [ ]:
TF02_arr=TF02.to_numpy()

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk
matrix_array=X.to_numpy()
NUM_CLUSTERS=10
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(TF02_arr, assign_clusters=True)
print (assigned_clusters)

In [ ]:
jaccard(assigned_clusters,True_cluster_arr)

In [ ]:
unique, counts = np.unique(assigned_clusters, return_counts=True)
dict(zip(unique, counts))